In [1]:
import sys
import os
import requests
import json
import csv
import decimal
import copy

#sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartenv/setup/challenges/util/__init__.py'>

In [2]:
!pwd

/smartenv/setup/challenges


In [3]:
SMARTENV_PATH = "/smartenv/"
GEN_CONFIG_PATH = "/smartenv/setup/generated_config/"
GLOBAL_INFO_PATH = GEN_CONFIG_PATH + "/global_info/"
GENESIS_JSON = "/smartenv/setup/environment/genesis_config/go-ethereum/berlin/genesis.json"

PARTICIPANTS_PATH = "/smartenv/participants/"
PARTICIPANTS_CSV = PARTICIPANTS_PATH + "participants.csv"
PP_CSV = PARTICIPANTS_CSV

SCRIPTS_PATH = "/smartenv/util/scripts/"
CHALLENGES_PATH = "/smartenv/challenges/"
CHL_NOTAWALLET_PATH = "/smartenv/challenges/notawallet/"

In [4]:
SRVIP = "172.18.0.8"
SRVPORT = "8545"
SRVNETVER = "1337"

In [5]:
SEED_VALUE = 16*10**18
SEED_VALUE_NOTAWALLET = 10*10**18
SEED_VALUE_BADPARITY = 30*10**18
SEED_VALUE_DAODOWN = 30*10**18
SEED_VALUE_FAILDICE = 30*10**18

# Challenge setup geth

## Start environment: smartenv-geth & smartenv-web3py 

We will run the server/geth node as well as the smar>tenv-web3py client docker containers on the same host machine. 

```bash
$ make run-smartenv-web3py # the container running this notebook
$ make run-smartenv-geth # the geth client this notebook connectes to
```


## Connect and check chain state

Note that for *PoA* nodes the middleware needs to be added.
Therefore, set the `poa` flag to true when connecting. 

In [20]:
#w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.1:8545"))
#w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True)
# check if connection was successful
assert w3.isConnected()

In [21]:
w3.api #web3 version used

'5.20.0'

In [22]:
w3.clientVersion # remote client version to which w3 is connected 

'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.15.4'

In [23]:
assert w3.net.version == SRVNETVER
w3.net.version

'1337'

In [24]:
w3.eth.blockNumber

17

In [25]:
w3.eth.getBlock(w3.eth.blockNumber)["gasLimit"]

14752937

In [26]:
w3.eth.mining

False

In [27]:
w3.eth.syncing

False

### Networ check

In [28]:
w3.net.peerCount

1

In [29]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://9d3297cebb326554af6e6d3146c19856b42a4e97f5a361bd51d8cdf66881ddc65ca54ba4082a2bcba236f1a18082ad3315fb2f5065c617dc7bead1d0c07b6f61@131.130.126.71:30303',
  'id': '11ee2e712a33a71f0c3cf7ee7cca0520c3b18584cda5ecd83ba8376a7248de64',
  'name': 'Geth/alice/v1.10.5-unstable-dde6f1e9-20210628/linux-amd64/go1.15.4',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.18.0.8:35260',
   'remoteAddress': '131.130.126.71:30303',
   'inbound': False,
   'trusted': False,
   'static': False}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 35,
    'head': '0x8286c0d997ab897c347d72ea6723ce8162af3fde28d4b0abb5b526b4e74ae57a'}),
   'snap': AttributeDict({'version': 1})})})]

In [30]:
w3.net.listening

True

In [31]:
w3.eth.accounts

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 '0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E',
 '0x959155655931D3398610a5ACDae41D7cD3A08912']

### Export (correct) enode info for server (docker container)

Delete the old generated config dir beforehand:

In [32]:
GEN_CONFIG_PATH

'/smartenv/setup/generated_config/'

In [33]:
!mkdir -p {GEN_CONFIG_PATH} 

In [34]:
poa_node_con = False
enode_file = GLOBAL_INFO_PATH + "/enodes"
# only needed if running directly on PoA node server:
#invalid_ip_port = "172.18.0.5:30303"
invalid_ip_port = "127.0.0.1:30303"
valid_ip_port = "128.131.169.35:30303"

if poa_node_con:
    if w3.clientVersion.startswith('Parity'):
        enode = w3.parity.enode()
    elif w3.clientVersion.startswith('Geth'):
        enode = w3.geth.admin.nodeInfo()['enode']
    enode = enode.replace(invalid_ip_port,valid_ip_port)
else:
    enode = w3.geth.admin.peers()[0]["enode"]
enode

'enode://9d3297cebb326554af6e6d3146c19856b42a4e97f5a361bd51d8cdf66881ddc65ca54ba4082a2bcba236f1a18082ad3315fb2f5065c617dc7bead1d0c07b6f61@131.130.126.71:30303'

In [35]:
# Write enode to file:
!mkdir -p $GLOBAL_INFO_PATH
!echo $enode > $enode_file

In [36]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000033f4f5ac17d677e188ab8d43149717632f9960d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0xa5f0a078d24ba19a4f4f01b77aa1d13d965b95dfbdca6ba2fa79e711d481272e'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

Copy password file over for generated accounts:

In [37]:
!cp {SMARTENV_PATH}/passwordfile {GLOBAL_INFO_PATH}

Copy `genesis.json` over for generated config:

In [38]:
!cp {GENESIS_JSON} {GLOBAL_INFO_PATH}

## Import students

In [39]:
print(PARTICIPANTS_PATH)
print(PARTICIPANTS_CSV)
print(PP_CSV)

/smartenv/participants/
/smartenv/participants/participants.csv
/smartenv/participants/participants.csv


In [40]:
!ls -la {PARTICIPANTS_CSV}

lrwxrwxrwx 1 smartenv smartenv 24 Jun 28 18:43 /smartenv/participants/participants.csv -> participants_current.csv


In [41]:
!head -n2 $PARTICIPANTS_CSV

Aljosha,Judmayer,ajudmayer@sba-research.org,1
Nicholas,Stifter,nstifter@sba-research.org,2


In [42]:
!file $PARTICIPANTS_CSV # get file format/encoding

/smartenv/participants/participants.csv: symbolic link to participants_current.csv


In [43]:
!wc -l $PARTICIPANTS_CSV

33 /smartenv/participants/participants.csv


In [44]:
wc_out = !wc -l $PARTICIPANTS_CSV # check number of stutend and compare later 
pp_count_wc = int(wc_out[0].split(" ")[0])
pp_count_wc

33

In [45]:
pp_list = list() # list of participants
pp_count = 0 # number of participants

in_fieldnames = ['firstname', 'lastname','email','ppid'] # ppid could be matr.

In [46]:
def read_students(path,pp_list):
    with open(path,'r',encoding='utf-8') as f_input:
    #with open(path,'r',encoding='utf-16') as f_input:
    #with open(path,'r',encoding='latin-1') as f_mail:
        #sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter='\t',quotechar='\"')
        sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter=',',quotechar='\"')
        for i,row in enumerate(sreader):
            #print(i,row)
            pp_dict = dict()
            for field in in_fieldnames:
                    pp_dict[field]=row[field]
            pp_list.append(pp_dict)
            #print(students_list)
    pp_count = i+1
    return pp_count

In [47]:
read_students(PP_CSV,pp_list)

33

In [48]:
pp_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1'}

In [49]:
pp_list[-1]

{'firstname': 'stub10',
 'lastname': 'stub10',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '123'}

In [50]:
# pp_list # long output

In [51]:
pp_count = len(pp_list)
assert pp_count == pp_count_wc
pp_count

33

## Create keys

In [52]:
SCRIPTS_PATH

'/smartenv/util/scripts/'

In [53]:
!cat {SCRIPTS_PATH}create_accounts.sh 

#!/bin/bash
#
# Run from root of repository

set -e

if [ -z "${1}" ] && [ -z "${PARTICIPANTS+x}" ];
then
    echo "PARTICIPANTS = No number given e.g., '42'"
    exit 2
elif [ ! -z "${1}" ];
then
    export PARTICIPANTS="${1}"
fi
echo "PARTICIPANTS = ${PARTICIPANTS}"

if [ -z "${DATADIR+x}" ];
then
    echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., './datadir/alice/'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

if [ -z "${TARGETDIR+x}" ];
then
        echo "TARGETDIR environment variable missing"
	echo "No Datadir given e.g., './setup/generated_config/keystore'"
	exit 2
fi
echo "TARGETDIR = ${TARGETDIR}"

mkdir -p ${TARGETDIR}

#for i in {0..90}};
for i in $(seq 1 ${PARTICIPANTS});
do 
	echo $i
	bash ./util/scripts/geth_account.sh new
done

cp -r "${DATADIR}/keystore" ${TARGETDIR}


In [54]:
!cat {SCRIPTS_PATH}geth_account.sh

#!/bin/bash
#
#
# Run with docker from root of repository:
# $ PWFILE=./passwordfile DATADIR=./datadir/accountmgt/ bash geth_account.sh new

set -e

if [ -z "${PWFILE+x}" ];
then
	echo "PWFILE environment variable missing"
	echo "No Password file given using default './passwordfile'"
	export PWFILE='./passwordfile'
fi
echo "PWFILE = ${PWFILE}"

if [ -f "${PWFILE}" ];
then
	echo "using stored password ... "	
else
	pwgen -s -B 32 1 > ${PWFILE}
fi

if [ -z "${DATADIR+x}" ];
then
        echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., './datadir/bob'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

mkdir -p ${DATADIR}

# run container without network
# just to generate and manage keys
docker run \
	--mount type=bind,source=$(pwd),target=/smartenv \
	-it smartenv-geth:latest \
		geth \
		--datadir "${DATADIR}" \
		--nodiscover \
		--maxpeers 0 \
		--ipcdisable \
		--verbosity 6  \
		--password "${PWFILE}" \
		account "$1" "$2" "$3" 



Run the following command from the root folder of the repository: 
(Take care that there are not more keys under the keystore in DATADIR or the TARGETDIR)

## Assign addresses to users (testaccounts & students)

In [55]:
#### CHANGE ####
 
# path where the raw ethereuem keys are located:
#source_path="./test/keystore/" 
source_path= GEN_CONFIG_PATH + "/keystore"

# Either run with test users or with all students:
#users_list = copy.deepcopy(testers_list)
users_list = copy.deepcopy(pp_list)

#users_list = copy.deepcopy(testers_list)
#users_list.extend(copy.deepcopy(students_list)

#### CHANGE END ####

In [56]:
# print some user as test before assignment
print(len(users_list))
users_list[0]

33


{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1'}

In [57]:
!ls {GEN_CONFIG_PATH}

global_info  keystore


In [58]:
!cd $source_path && pwd && ls | head -n1

/smartenv/setup/generated_config/keystore
UTC--2021-06-29T11-41-06.504958819Z--d305c767e93b6c714e8787bc0805cb10a5f1eb94


In [59]:
#!cd $source_path && ls ./ | head -n1 | xargs -Irpl cat rpl

In [60]:
def read_keypairs(path):
    """read keypairs from keystore
    """
    _keypairs = list()
    # read out keystore folder 
    for root,dirs,files in os.walk(path):
        #print("root = {}".format(root))
        #print("dirs = {}".format(dirs))
        for f in files:
            with open(root + "/" + f,"r",encoding="utf-8") as f_input:
                data = json.loads(f_input.read()) # load all key pair data from file
                data["filename"] = f # add file name to data for later
                _keypairs.append(data)
    return _keypairs   

In [61]:
def assign_addresses(keystore_path,users,attach_path="/smartenv/setup/generated_config/attach"):
    """ map keypairs to users and add info to user dict
    This also creates the attach directory. 
    """
    keypairs = read_keypairs(keystore_path)
    assert len(keypairs) >= len(users)
    
    for u in users:
        kp = keypairs.pop()
        u["account"] = kp["address"]
        #u["keypair"] = kp
        #u["key"] = kp["crypto"]["ciphertext"]
        u["filename"] = kp["filename"]
        
        arg_dest = attach_path + "/" + u["account"]
        !mkdir -p $arg_dest
        arg_src = keystore_path + "/" + kp["filename"]
        !cp $arg_src $arg_dest
        u["attach"] = "/tmp/attach/" + u["account"] + ".tar.gz"
        print(".",end=".")
    return users

In [62]:
attach_path= GEN_CONFIG_PATH + "/attach"
#!rm -rf attach_path
assign_addresses(source_path,users_list,attach_path)

# print some user as test after assignment
users_list[0]

..................................................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz'}

In [63]:
def getBalance(address):
    address = w3.toChecksumAddress(address)
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [64]:
getBalance(users_list[0]["account"])

0

In [65]:
w3.eth.accounts

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 '0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E',
 '0x959155655931D3398610a5ACDae41D7cD3A08912']

In [66]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6 :  1000000
0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E :  0
0x959155655931D3398610a5ACDae41D7cD3A08912 :  0


In [67]:
w3.eth.defaultAccount = w3.eth.accounts[0]

In [68]:
w3.eth.defaultAccount

'0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6'

In [69]:
def send_accounts_money(users,amount):
    for u in users:
        account = w3.toChecksumAddress(u["account"])
        u["seedbalance"] = amount
        w3.eth.sendTransaction({"from":w3.eth.defaultAccount,"to":account,"value":amount})
    return

In [70]:
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"notforproductiveuse",0)

True

In [71]:
send_accounts_money(users_list,SEED_VALUE)

In [72]:
!sleep 16
getBalance(users_list[0]["account"])

Decimal('16')

In [73]:
getBalance(users_list[-1]["account"])

Decimal('16')

In [74]:
def check_accounts_money(users,predicate=(lambda a, b: a == b)):
    for u in users:
        u_balance = w3.eth.getBalance(w3.toChecksumAddress(u["account"]))
        u["balance"] = u_balance
        u_seedbalance = u["seedbalance"]
        if predicate(u_balance,u_seedbalance) == False:
            print("Predicate violation for:\n",u)
            assert False,"Predicate violation"

In [75]:
check_accounts_money(users_list)

In [76]:
check_accounts_money(users_list,predicate=(lambda a, b: a >= b))
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

### Test manual account handling

### Deployment functions

In [77]:
sys.path.append( SMARTENV_PATH + "/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartenv/setup/challenges/util/__init__.py'>

In [78]:
def forall_compile_and_deploy(users,
                              path,
                              contract_name,
                              value,
                              force_argument=None,
                              argument2=None,
                              compiler="solc"):
    print("Started for {} users:".format(len(users)))
    for u in users:
        if force_argument is not None:
            print("*",end="")
            #print("Force argument: {}".format(force_argument))
            u_addr = force_argument
        else:
            u_addr = w3.toChecksumAddress(u["account"])
            
        if argument2 is not None and argument2 is True:
            print("~",end="")
            #print("Double sha3 argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=w3.toInt(hexstr=w3.sha3(text=w3.sha3(text=u_addr).hex()).hex()),
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        elif argument2 is not None and isinstance(argument2,type(str())):
            print("+",end="")
            #print("Custom argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=argument2,
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        else:
            print(".",end="")
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        # returns HexBytes
        # import hexbytes
        # help(hexbytes.main)
        u[contract_name + "_tx"] = tx_hash.hex()
        u[contract_name + "_seedvalue"] = value
        
    for u in users:
        tx_receipt = w3.eth.waitForTransactionReceipt(u[contract_name + "_tx"])
        u[contract_name + "_addr"] = tx_receipt['contractAddress']
        print("-",end="")
        
    return users

In [79]:
def forall_check_contract(users,
                          c_path,
                          c_name,
                          predicate=(lambda a, b: a == b),
                          compiler="solc"):
    # Check for all users if the contract is initialized to the respective user
    # and that the balance is the seed value
    print("Started for {} users:".format(len(users)))
    for u in users:
        c_abi = util.compile_contract_with_libs(compiler_path=compiler,src_path=c_path)["abi"]
        u_c_addr = u[c_name + "_addr"]
        u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=False)
    
        # check owner
        #print(u_c_instance.functions.getStudent().call())
        #print(w3.toChecksumAddress(u["account"]))
        assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(u["account"])   
        # check balance
        u_c_seedvalue = u[c_name + "_seedvalue"]
        u_c_balance = w3.eth.getBalance(u_c_addr)   
        u[c_name + "_balance"] = u_c_balance
        if predicate(u_c_balance,u_c_seedvalue) == False:
            print("Predicate violation for:\n",u)
        print(".",end="")
    return users

In [80]:
def copy_challenge_metadata(users,challenge_path,challenge_name,attach_path="./attach"):
    # copy challenge metadata to individual user folders helper function
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_dest_dir = attach_path + "/" + d + "/" + challenge_name
            print(arg_dest_dir)
            !mkdir -p $arg_dest_dir
            !cp -r $challenge_path $arg_dest_dir

# Challange: notawallet 

In [81]:
CHL_NOTAWALLET_PATH

'/smartenv/challenges/notawallet/'

In [82]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [83]:
# users_list[-1]
users_list[1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nstifter@sba-research.org',
 'ppid': '2',
 'account': '1375859c41590c8ad41f1ff086e3d4bb8f5ea1d9',
 'filename': 'UTC--2021-06-29T11-41-08.853492719Z--1375859c41590c8ad41f1ff086e3d4bb8f5ea1d9',
 'attach': '/tmp/attach/1375859c41590c8ad41f1ff086e3d4bb8f5ea1d9.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [84]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [85]:
!solc --overwrite -o "{CHL_NOTAWALLET_PATH}/publish/" --abi --bin "{CHL_NOTAWALLET_PATH}/publish/NotAWallet.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges/notawallet//publish/.


In [86]:
## Connections refresh maybe needed
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True) 
assert w3.isConnected()

In [87]:
path= CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol"
challange_name="notawallet"
seedvalue=10*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 33 users:
.................................---------------------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375'}

In [88]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [89]:
c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path=CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol")["abi"]
c_abi

[{'inputs': [{'internalType': 'address',
    'name': '_student',
    'type': 'address'}],
  'stateMutability': 'payable',
  'type': 'constructor'},
 {'inputs': [{'internalType': 'address',
    'name': 'newowner',
    'type': 'address'}],
  'name': 'addOwner',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'deposit',
  'outputs': [],
  'stateMutability': 'payable',
  'type': 'function'},
 {'inputs': [],
  'name': 'getStudent',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'address',
    'name': 'testowner',
    'type': 'address'}],
  'name': 'isOwner',
  'outputs': [{'internalType': 'bool', 'name': '', 'type': 'bool'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'owner',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'functi

In [90]:
w3.toChecksumAddress(users_list[0]["account"])

'0x032FcB0a1c233Da1546d2242Ce0b4C4A51118Ec3'

In [91]:
notawallet_addr = users_list[0]["notawallet_addr"]
notawallet_addr

'0x5341d88F2CC84e9Ca4F420Fd70627C7669570375'

In [92]:
u_c_instance = util.get_contract_instance(notawallet_addr,c_abi,concise=False)

In [93]:
assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(users_list[0]["account"])
u_c_instance.functions.getStudent().call()

'0x032FcB0a1c233Da1546d2242Ce0b4C4A51118Ec3'

In [94]:
path=CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol"
challange_name="notawallet"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 33 users:
.................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000}

In [95]:
# copy challenge metadata
challenge_path= CHL_NOTAWALLET_PATH + "/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="notawallet"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/dfa41679d35235afc79907088d76977fecc37474/notawallet
/smartenv/setup/generated_config//attach/56ba19f6813ff3939873eb529d4e12e0991e3a00/notawallet
/smartenv/setup/generated_config//attach/93d88d905cdc33ffb5766c30e87d87be43ba1b0f/notawallet
/smartenv/setup/generated_config//attach/f5ff9539dd171d351ee65d713a63989181b63df9/notawallet
/smartenv/setup/generated_config//attach/bb872acc4ee3d07836e6cdafb15c8fc2f9dcaf63/notawallet
/smartenv/setup/generated_config//attach/478c6cf7c49203e4c69fee138372f8731dee36e1/notawallet
/smartenv/setup/generated_config//attach/d4a1e16aae7a5f75e212cc3805b644074666c235/notawallet
/smartenv/setup/generated_config//attach/d305c767e93b6c714e8787bc0805cb10a5f1eb94/notawallet
/smartenv/setup/generated_config//attach/f7c4f0fa72f6376cde89984109bd6d75e73e8f24/notawallet
/smartenv/setup/generated_config//attach/beb57567e5a962451e8433618f787eca7e695cf5/notawallet
/smartenv/setup/generated_config//attach/836afc53b26c45aa67850b9729dba

# Challenge: badparity

In [96]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000}

In [97]:
!cat "{CHALLENGES_PATH}/badparity/publish/Wallet.sol"

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Wallet {
    address walletLibrary;
    address payable owner;
    address payable student;

    event LogValue(uint256 exitcode,uint256 amount);

    // constructor, called once when this contract is created 
    constructor(address payable _student, address lib) payable {
        student = _student;  
        walletLibrary = lib; // hardcode lib address at deploy time
        // init the owner with the respective lib contract
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
    }

    function getOwner() public view returns (address payable) {
        return owner;
    } 

    function getWalletLibrary() public view returns (address) {
        return walletLibrary;
    }

    function withdraw(uint256 amount) public returns (bool) {
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
        if ( suc

In [98]:
!solc --overwrite -o "{CHALLENGES_PATH}/badparity/publish/" --abi --bin "{CHALLENGES_PATH}/badparity/publish/Wallet.sol"

 --> /smartenv/challenges//badparity/publish/Wallet.sol:4:1:
  |
4 | contract Wallet {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> /smartenv/challenges//badparity/publish/Wallet.sol:47:5:
   |
47 |     fallback() external payable {
   |     ^ (Relevant source part starts here and spans across multiple lines).

  --> /smartenv/challenges//badparity/publish/Wallet.sol:16:9:
   |
16 |         walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
   |         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  --> /smartenv/challenges//badparity/publish/Wallet.sol:50:9:
   |
50 |         walletLibrary.delegatecall(msg.data);
   |         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  --> /smartenv/challenges//badparity/publish/Wallet.sol:28:24:
   |
28 |         (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWith

In [99]:
!solc --overwrite -o "{CHALLENGES_PATH}/badparity/publish/" --abi --bin "{CHALLENGES_PATH}/badparity/publish/WalletLibrary.sol"

 --> /smartenv/challenges//badparity/publish/WalletLibrary.sol:4:1:
  |
4 | contract WalletLibrary {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> /smartenv/challenges//badparity/publish/WalletLibrary.sol:36:6:
   |
36 |      fallback() external payable {
   |      ^ (Relevant source part starts here and spans across multiple lines).



In [100]:
# compile and deploy lib once for all users
path=CHALLENGES_PATH + "/badparity/publish/WalletLibrary.sol"

lib = util.compile_and_deploy_contract(path,
                                 patch_api=False,
                                 concise=False)
lib.address

'0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0'

In [101]:
path=CHALLENGES_PATH + "/badparity/publish/Wallet.sol"
challange_name="badparity"
seedvalue=SEED_VALUE_BADPARITY

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=lib.address)

for u in users_list:
    u[challange_name + "_lib"] = lib.address

users_list[0]

Started for 33 users:
+++++++++++++++++++++++++++++++++---------------------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0'}

In [102]:
# spot check of one contract
c_abi = util.compile_contract_with_libs("solc",path)["abi"]
u_c_addr = users_list[0][challange_name + "_addr"]
u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=True)

assert users_list[0][challange_name + "_lib"] == lib.address
    
# check lib
assert str(u_c_instance.getWalletLibrary()).lower() == str(lib.address).lower()
assert str(u_c_instance.getOwner()).lower() == str(w3.eth.accounts[0]).lower()

# check balance
assert w3.eth.getBalance(u_c_addr) == SEED_VALUE_BADPARITY

In [103]:
u_c_instance.getOwner()

'0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6'

In [104]:
u_c_instance.getStudent()

'0x032FcB0a1c233Da1546d2242Ce0b4C4A51118Ec3'

In [105]:
w3.parity.personal.listAccounts()

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 '0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E',
 '0x959155655931D3398610a5ACDae41D7cD3A08912']

In [106]:
# copy challenge metadata
challenge_path=CHALLENGES_PATH + "/badparity/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="badparity"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/dfa41679d35235afc79907088d76977fecc37474/badparity
/smartenv/setup/generated_config//attach/56ba19f6813ff3939873eb529d4e12e0991e3a00/badparity
/smartenv/setup/generated_config//attach/93d88d905cdc33ffb5766c30e87d87be43ba1b0f/badparity
/smartenv/setup/generated_config//attach/f5ff9539dd171d351ee65d713a63989181b63df9/badparity
/smartenv/setup/generated_config//attach/bb872acc4ee3d07836e6cdafb15c8fc2f9dcaf63/badparity
/smartenv/setup/generated_config//attach/478c6cf7c49203e4c69fee138372f8731dee36e1/badparity
/smartenv/setup/generated_config//attach/d4a1e16aae7a5f75e212cc3805b644074666c235/badparity
/smartenv/setup/generated_config//attach/d305c767e93b6c714e8787bc0805cb10a5f1eb94/badparity
/smartenv/setup/generated_config//attach/f7c4f0fa72f6376cde89984109bd6d75e73e8f24/badparity
/smartenv/setup/generated_config//attach/beb57567e5a962451e8433618f787eca7e695cf5/badparity
/smartenv/setup/generated_config//attach/836afc53b26c45aa67850b9729dbad5d1f53f1b

# Challenge: Daodown

In [107]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0'}

In [108]:
# !solc --overwrite -o "{CHALLENGES_PATH}/daodown/publish/" --abi --bin "{CHALLENGES_PATH}/daodown/publish/EDao.sol"

In [109]:
!solc_7.4 --overwrite -o "{CHALLENGES_PATH}/daodown/publish/" --abi --bin "{CHALLENGES_PATH}/daodown/publish/EDao.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges//daodown/publish/.


In [110]:
path=CHALLENGES_PATH + "/daodown/publish/EDao.sol"
challange_name="daodown"
seedvalue=SEED_VALUE_DAODOWN

forall_compile_and_deploy(users_list,
                          path,
                          challange_name,
                          seedvalue,
                          compiler="solc_7.4")
users_list[0]

Started for 33 users:
.................................---------------------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [111]:
w3.eth.blockNumber

60

In [112]:
w3.eth.getBlock(w3.eth.blockNumber)["gasUsed"]

3515863

In [113]:
w3.eth.getBlock(w3.eth.blockNumber-3)["gasUsed"]

0

In [114]:
path=CHALLENGES_PATH + "/daodown/publish/EDao.sol"
challange_name="daodown"
forall_check_contract(users_list,
                      path,
                      challange_name,
                      compiler="solc_7.4")
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 33 users:
.................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [115]:
# copy challenge metadata
challenge_path=CHALLENGES_PATH + "/daodown/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="daodown"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/dfa41679d35235afc79907088d76977fecc37474/daodown
/smartenv/setup/generated_config//attach/56ba19f6813ff3939873eb529d4e12e0991e3a00/daodown
/smartenv/setup/generated_config//attach/93d88d905cdc33ffb5766c30e87d87be43ba1b0f/daodown
/smartenv/setup/generated_config//attach/f5ff9539dd171d351ee65d713a63989181b63df9/daodown
/smartenv/setup/generated_config//attach/bb872acc4ee3d07836e6cdafb15c8fc2f9dcaf63/daodown
/smartenv/setup/generated_config//attach/478c6cf7c49203e4c69fee138372f8731dee36e1/daodown
/smartenv/setup/generated_config//attach/d4a1e16aae7a5f75e212cc3805b644074666c235/daodown
/smartenv/setup/generated_config//attach/d305c767e93b6c714e8787bc0805cb10a5f1eb94/daodown
/smartenv/setup/generated_config//attach/f7c4f0fa72f6376cde89984109bd6d75e73e8f24/daodown
/smartenv/setup/generated_config//attach/beb57567e5a962451e8433618f787eca7e695cf5/daodown
/smartenv/setup/generated_config//attach/836afc53b26c45aa67850b9729dbad5d1f53f1b1/daodown
/smartenv/

## Challenge: Faildice

In [116]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [117]:
!ls -la /usr/local/bin/solc*

-rwxr-xr-x 1 root root 11833912 Jun 15 13:35 /usr/local/bin/solc
-rwxr-xr-x 1 root root  6284040 Feb 12  2019 /usr/local/bin/solc_5.4
-rwxr-xr-x 1 root root 11192752 Oct 19  2020 /usr/local/bin/solc_7.4
-rwxr-xr-x 1 root root 11833912 Apr 21 16:53 /usr/local/bin/solc_8.4


In [118]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [119]:
!solc --overwrite -o "{CHALLENGES_PATH}/faildice/publish/" --abi --bin "{CHALLENGES_PATH}/faildice/publish/SatoshiFailDice.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges//faildice/publish/.


In [120]:
path= CHALLENGES_PATH + "/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
seedvalue=SEED_VALUE_FAILDICE

forall_compile_and_deploy(users_list,
                          path,
                          challange_name,
                          seedvalue,
                          argument2=True)
users_list[0]

Started for 33 users:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~---------------------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [121]:
path= CHALLENGES_PATH + "/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 33 users:
.................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [122]:
# copy challenge metadata
challenge_path= CHALLENGES_PATH + "/faildice/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="faildice"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/dfa41679d35235afc79907088d76977fecc37474/faildice
/smartenv/setup/generated_config//attach/56ba19f6813ff3939873eb529d4e12e0991e3a00/faildice
/smartenv/setup/generated_config//attach/93d88d905cdc33ffb5766c30e87d87be43ba1b0f/faildice
/smartenv/setup/generated_config//attach/f5ff9539dd171d351ee65d713a63989181b63df9/faildice
/smartenv/setup/generated_config//attach/bb872acc4ee3d07836e6cdafb15c8fc2f9dcaf63/faildice
/smartenv/setup/generated_config//attach/478c6cf7c49203e4c69fee138372f8731dee36e1/faildice
/smartenv/setup/generated_config//attach/d4a1e16aae7a5f75e212cc3805b644074666c235/faildice
/smartenv/setup/generated_config//attach/d305c767e93b6c714e8787bc0805cb10a5f1eb94/faildice
/smartenv/setup/generated_config//attach/f7c4f0fa72f6376cde89984109bd6d75e73e8f24/faildice
/smartenv/setup/generated_config//attach/beb57567e5a962451e8433618f787eca7e695cf5/faildice
/smartenv/setup/generated_config//attach/836afc53b26c45aa67850b9729dbad5d1f53f1b1/faildice

## Add optional/additional meta data

In [123]:
def add_login_token(path,users):
    login_tokens=list()
    with open(path,'r') as f_input:
        for line in f_input:
            #print(line.strip())
            login_tokens.append(line.strip())    
    
    for i,u in enumerate(users):
        u["login_token"] = login_tokens[i]
    
    return login_tokens.copy()

In [124]:
login_tokens = add_login_token(PARTICIPANTS_PATH + "login_tokens",users_list)

In [125]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [126]:
# login_tokens

## Write Students

In [127]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [128]:
import json

def persist_participants_json(path,users):
    with open(path,'w') as f_output:
        json.dump(users,f_output)

In [129]:
out_file_json = GEN_CONFIG_PATH + "/participants_meta.json"
persist_participants_json(out_file_json,users_list)
len(users_list)

33

In [130]:
import csv

def persist_participants_csv(path,
                   out_fieldnames=['firstname', 'lastname','ppid','email','cc','attach','contract','account'],
                   cc_mail="ajudmayer@sba-research.org"):
    with open(path,'w',encoding='utf-8') as f_output:
    #with open(path,'w',encoding='latin-1') as f_output:
        writer = csv.DictWriter(f_output, 
                                fieldnames=out_fieldnames, 
                                delimiter=',',
                                quotechar='\"',
                                extrasaction='ignore')
        writer.writeheader()
        for i,row in enumerate(users_list):
            row["cc"]=cc_mail
            #print(row)
            writer.writerow(row)
    return i+1

In [131]:
out_file = GEN_CONFIG_PATH + "/participants_meta.csv"
out_fieldnames = ['ppid','email','cc','attach','account']
#out_fieldnames = ['firstname', 'lastname','matr','email','cc','attach','contract','account']
#out_fieldnames = ['firstname', 
#                  'lastname',
#                  'matr',
#                  'email',
#                  'cc',
#                  'attach',
#                  'account',
#                  'notawallet_addr',
#                 ]
cc_mail = "ajudmayer@sba-research.org"
persist_participants_csv(out_file,out_fieldnames,cc_mail)

33

In [132]:
!head -n2 $out_file

In [133]:
!wc -l $out_file

34 /smartenv/setup/generated_config//participants_meta.csv


## Prepare mail

In [134]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo

In [135]:
import os

def dump_addresses(users,attach_path=GEN_CONFIG_PATH + "/attach"):
    for u in users:
        arg_dest = attach_path + "/" + u["account"] + "/" + "addresses"
        ch_string = "notawallet_addr = " + u["notawallet_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "badparity_addr = " + u["badparity_addr"]
        !echo $ch_string >> $arg_dest    
        
        ch_string = "daodown_addr = " + u["daodown_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "faildice_addr = " + u["faildice_addr"]
        !echo $ch_string >> $arg_dest
        
        arg_dest = attach_path + "/" + u["account"] + "/" + "login_token"
        ch_string = "login_token = " + u["login_token"]
        !echo $ch_string >> $arg_dest
        
        print(".",end="")

In [136]:
# write all addresses to folders 
dump_addresses(users_list,attach_path=GEN_CONFIG_PATH + "/attach")

.................................

In [137]:
# copy general information i.e., datadir to individual folders
source_path=GLOBAL_INFO_PATH
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name=""
copy_challenge_metadata(users_list,source_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/dfa41679d35235afc79907088d76977fecc37474/
/smartenv/setup/generated_config//attach/56ba19f6813ff3939873eb529d4e12e0991e3a00/
/smartenv/setup/generated_config//attach/93d88d905cdc33ffb5766c30e87d87be43ba1b0f/
/smartenv/setup/generated_config//attach/f5ff9539dd171d351ee65d713a63989181b63df9/
/smartenv/setup/generated_config//attach/bb872acc4ee3d07836e6cdafb15c8fc2f9dcaf63/
/smartenv/setup/generated_config//attach/478c6cf7c49203e4c69fee138372f8731dee36e1/
/smartenv/setup/generated_config//attach/d4a1e16aae7a5f75e212cc3805b644074666c235/
/smartenv/setup/generated_config//attach/d305c767e93b6c714e8787bc0805cb10a5f1eb94/
/smartenv/setup/generated_config//attach/f7c4f0fa72f6376cde89984109bd6d75e73e8f24/
/smartenv/setup/generated_config//attach/beb57567e5a962451e8433618f787eca7e695cf5/
/smartenv/setup/generated_config//attach/836afc53b26c45aa67850b9729dbad5d1f53f1b1/
/smartenv/setup/generated_config//attach/fe3ebfca70a4b1ced2f560950d6a447a5985e794/
/sma

In [138]:
# create archives
import os

def create_user_archives(users,attach_path=GEN_CONFIG_PATH + "/attach",attach_tgz=GEN_CONFIG_PATH + "/attach_tgz"):
    !mkdir -p $attach_tgz
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_src_dir = attach_path + "/" + d 
            arg_dst_dir = attach_tgz + "/" + d + ".tar.gz"
            print(arg_dst_dir)
            #With verbose output:
            #!tar -cvzf $arg_dst_dir -C $arg_src_dir . 
            #Without verbose output
            !tar -czf $arg_dst_dir -C $arg_src_dir . 

In [139]:
attach_path=GEN_CONFIG_PATH + "/attach"
attach_tgz=GEN_CONFIG_PATH + "/attach_tgz"
create_user_archives(users_list,attach_path,attach_tgz)

/smartenv/setup/generated_config//attach_tgz/dfa41679d35235afc79907088d76977fecc37474.tar.gz
/smartenv/setup/generated_config//attach_tgz/56ba19f6813ff3939873eb529d4e12e0991e3a00.tar.gz
/smartenv/setup/generated_config//attach_tgz/93d88d905cdc33ffb5766c30e87d87be43ba1b0f.tar.gz
/smartenv/setup/generated_config//attach_tgz/f5ff9539dd171d351ee65d713a63989181b63df9.tar.gz
/smartenv/setup/generated_config//attach_tgz/bb872acc4ee3d07836e6cdafb15c8fc2f9dcaf63.tar.gz
/smartenv/setup/generated_config//attach_tgz/478c6cf7c49203e4c69fee138372f8731dee36e1.tar.gz
/smartenv/setup/generated_config//attach_tgz/d4a1e16aae7a5f75e212cc3805b644074666c235.tar.gz
/smartenv/setup/generated_config//attach_tgz/d305c767e93b6c714e8787bc0805cb10a5f1eb94.tar.gz
/smartenv/setup/generated_config//attach_tgz/f7c4f0fa72f6376cde89984109bd6d75e73e8f24.tar.gz
/smartenv/setup/generated_config//attach_tgz/beb57567e5a962451e8433618f787eca7e695cf5.tar.gz
/smartenv/setup/generated_config//attach_tgz/836afc53b26c45aa67850b972

## Test loading a key

## Load Participants again and attach meta data

In [142]:
import json

def load_users(path):
    with open(path,'r') as f_input:
        return json.load(f_input)

In [143]:
users_loaded = load_users(GEN_CONFIG_PATH + "/participants_meta.json")
users_loaded[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo